In [1]:
# train_model_with_features.ipynb – Kết hợp Sentence-BERT + đặc trưng phụ để huấn luyện
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb
import joblib
from tqdm.notebook import tqdm

In [2]:
# Load dữ liệu và đặc trưng phụ
print("📂 Đang đọc dữ liệu và đặc trưng phụ...")
df = pd.read_csv("cleaned_data_with_features.csv")
X_extra = np.load("X_features_scaled.npy")

# Load Sentence-BERT và tạo embedding
print("🔍 Đang tạo embedding từ clean_text với Sentence-BERT...")
model = SentenceTransformer("all-MiniLM-L6-v2")
X_embed = model.encode(df['clean_text'].tolist(), show_progress_bar=True)

# Gộp embedding + đặc trưng phụ
print("🧩 Đang kết hợp embedding và đặc trưng phụ...")
X = np.hstack((X_embed, X_extra))
y = df['label'].values

# Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

📂 Đang đọc dữ liệu và đặc trưng phụ...
🔍 Đang tạo embedding từ clean_text với Sentence-BERT...


Batches:   0%|          | 0/2804 [00:00<?, ?it/s]

🧩 Đang kết hợp embedding và đặc trưng phụ...


In [3]:
# Huấn luyện mô hình
print("⚙️ Đang huấn luyện mô hình XGBoost...")
clf = xgb.XGBClassifier(eval_metric='logloss')
clf.fit(X_train, y_train)

# Đánh giá
print("📊 Kết quả mô hình:")
y_pred = clf.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Lưu mô hình
joblib.dump(clf, "xgb_model_with_features.joblib")
print("💾 Mô hình đã lưu vào xgb_model_with_features.joblib")


⚙️ Đang huấn luyện mô hình XGBoost...
📊 Kết quả mô hình:
✅ Accuracy: 0.9938694755615003
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      9359
           1       0.99      0.99      0.99      8584

    accuracy                           0.99     17943
   macro avg       0.99      0.99      0.99     17943
weighted avg       0.99      0.99      0.99     17943

💾 Mô hình đã lưu vào xgb_model_with_features.joblib
